Міністерство освіти і науки України
Національний технічний університет України
"Київський політехнічний інститут імені Ігоря Сікорського"
Фізико-технічний інститут



                                                ПІДГОТОВКА ТА АНАЛІЗ ДАНИХ
                                                Лабораторна робота № 5
                                                Візуалізація даних







                                                                                                             Виконав:
                                                                                                            студент гр. ФБ-24 Берещук Р.Р.

Завдання 1

In [17]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, CheckButtons

def harmonic(tm, amplitude, frequency, phase):
    return amplitude * np.sin(2 * np.pi * frequency * tm + phase)

def create_noise(tm, noise_mean, noise_covariance):
    return np.random.normal(noise_mean, np.sqrt(noise_covariance), len(tm))

def harmonic_with_noise(tm, amplitude, frequency, phase=0, noise_mean=0, noise_covariance=0.1, show_noise=None, noise=None):
    harmonic_signal = harmonic(tm, amplitude, frequency, phase)
    if noise is not None and show_noise:
        return harmonic_signal + noise
    elif show_noise:
        global noise_global
        noise_global = create_noise(tm, noise_mean, noise_covariance)
        return harmonic_signal + noise_global
    else:
        return harmonic_signal

def moving_avg(data, window_size):
    moving_avg = []
    window_size = int(window_size)
    for i in range(len(data)):
        if i < window_size:
            avg = np.mean(data[:i+1])
        else:
            avg = np.mean(data[i-window_size+1:i+1])
        moving_avg.append(avg)
    return moving_avg

initial_amplitude = 1.0
initial_frequency = 1.0
initial_phase = 0.0
initial_noise_mean = 0.0
initial_noise_covariance = 0.1
noise_global = None

tm = np.linspace(0, 10, 1000)
sampling_frequency = 1/(tm[1]-tm[0])

fig, ax = plt.subplots()
ax.set_xlim(tm[0], 10)
ax.set_ylim(-10, 10)
ax.grid(True)

plt.subplots_adjust(left=0.12, bottom=0.45, top=0.95)


pure_harmonic_line, = ax.plot(tm, harmonic(tm, initial_amplitude, frequency=initial_frequency, phase=initial_phase),
                               lw=2, color="violet", linestyle="--", label='Чиста гармоніка')

harmonic_line, = ax.plot(tm, harmonic_with_noise(tm, initial_amplitude, frequency=initial_frequency,
                                                 phase=initial_phase, noise_mean=initial_noise_mean,
                                                 noise_covariance=initial_noise_covariance),   
                          lw=3, color="red", label='Сигнал з шумом')

filtered_signal = moving_avg(harmonic_line.get_ydata(), sampling_frequency)
l_filtered, = ax.plot(tm, filtered_signal, lw=2, color='green', label='Відфільтрований сигнал')

ax.legend()

s_amplitude = Slider(ax=plt.axes([0.1, 0.35, 0.65, 0.03]), label="Амплітуда", valmin=0.1, valmax=10.0, valinit=initial_amplitude, valstep=0.1)
s_frequency = Slider(ax=plt.axes([0.1, 0.30, 0.65, 0.03]), label="Частота", valmin=0.1, valmax=10.0, valinit=initial_frequency, valstep=0.1)
s_phase = Slider(ax=plt.axes([0.1, 0.25, 0.65, 0.03]), label="Фаза", valmin=0.0, valmax=2 * np.pi, valinit=initial_phase, valstep=0.1)
s_noise_mean = Slider(ax=plt.axes([0.1, 0.20, 0.65, 0.03]), label="Амплітуда шуму", valmin=-1.0, valmax=1.0, valinit=initial_noise_mean, valstep=0.1)
s_noise_covariance = Slider(ax=plt.axes([0.1, 0.15, 0.65, 0.03]), label="Диспепрсія шуму", valmin=0.0, valmax=1.0, valinit=initial_noise_covariance, valstep=0.1)
s_cutoff_frequency = Slider(ax=plt.axes([0.1, 0.10, 0.65, 0.03]), label="Розмір", valmin=1, valmax=35, valinit=3, valstep=1)

ck_show_noise = CheckButtons(ax=plt.axes([0.1, 0.05, 0.1, 0.03]), labels=['Показати шум'], actives=[0])

button_regenerate_noise = Button(ax=plt.axes([0.25, 0.005, 0.1, 0.03]), label='Сгенерувати шум')
button_random_param = Button(ax=plt.axes([0.4, 0.005, 0.1, 0.03]), label='Рандомні параметри')
button_reset = Button(ax=plt.axes([0.55, 0.005, 0.1, 0.03]), label='Скинути')

def update(val):
    amplitude = s_amplitude.val
    frequency = s_frequency.val
    phase = s_phase.val
    noise_mean = s_noise_mean.val
    noise_covariance = s_noise_covariance.val

    global initial_noise_mean, initial_noise_covariance, noise_global

    if initial_noise_covariance != noise_covariance or initial_noise_mean != noise_mean:
        initial_noise_mean = noise_mean
        initial_noise_covariance = noise_covariance
        noise_global = create_noise(tm, noise_mean, noise_covariance)

    show_noise = ck_show_noise.get_status()[0]
    harmonic_line.set_ydata(harmonic_with_noise(tm, amplitude, frequency, phase, noise_mean, noise_covariance, show_noise, noise_global))

    cutoff_frequency = s_cutoff_frequency.val
    filtered_signal = moving_avg(harmonic_line.get_ydata(), cutoff_frequency)
    l_filtered.set_ydata(filtered_signal)

    fig.canvas.draw_idle()

    pure_harmonic_line.set_ydata(harmonic(tm, amplitude, frequency, phase))
    fig.canvas.draw_idle()

sliders = [s_amplitude, s_frequency, s_phase, s_noise_mean, s_noise_covariance, s_cutoff_frequency]
buttons = [ck_show_noise, button_regenerate_noise, button_random_param, button_reset]

def regenerate_noise(ch):
    s_noise_mean.set_val(np.random.uniform(-1.0, 1.0))
    s_noise_covariance.set_val(np.random.uniform(0.0, 1.0))
    update(None)

def random_params(ch):
    s_amplitude.set_val(np.random.uniform(0.1, 10.0))
    s_frequency.set_val(np.random.uniform(0.1, 10.0))
    s_phase.set_val(np.random.uniform(0.0, 2 * np.pi))
    s_noise_mean.set_val(np.random.uniform(-1.0, 1.0))
    s_noise_covariance.set_val(np.random.uniform(0.0, 1.0))
    update(None)

def reset_params(ch):
    for slider in sliders:
        slider.reset()
    ck_show_noise.set_active([0])
    update(None)

for slider in sliders:
    slider.on_changed(update)

for button in buttons:
    if isinstance(button, CheckButtons):
        button.on_clicked(update)
    else:
        if button.label.get_text() == 'Сгенерувати шум':
            button.on_clicked(regenerate_noise)
        elif button.label.get_text() == 'Рандомні параметри':
            button.on_clicked(random_params)
        elif button.label.get_text() == 'Скинути':
            button.on_clicked(reset_params)

plt.show()









Завдання 2

In [ ]:
from bokeh.plotting import figure, curdoc
from bokeh.layouts import column, row
from bokeh.models import Slider, Button, CheckboxGroup
import numpy as np

def harmonic(t, amplitude, frequency, phase):
    return amplitude * np.sin(2 * np.pi * frequency * t + phase)

def create_noise(t, noise_mean, noise_covariance):
    return np.random.normal(noise_mean, np.sqrt(noise_covariance), len(t))

def harmonic_with_noise(t, amplitude, frequency, phase=0, noise_mean=0, noise_covariance=0.1, show_noise=None, noise=None):
    harmonic_signal = harmonic(t, amplitude, frequency, phase)
    if noise is not None and show_noise:
        return harmonic_signal + noise
    elif show_noise:
        global noise_g
        noise_g = create_noise(t, noise_mean, noise_covariance)
        return harmonic_signal + noise_g
    else:
        return harmonic_signal

def moving_avg(data, window_size):
    moving_avg = []
    window_size = int(window_size)
    for i in range(len(data)):
        if i < window_size:
            avg = np.mean(data[:i+1])
        else:
            avg = np.mean(data[i-window_size+1:i+1])
        moving_avg.append(avg)
    return moving_avg

initial_amplitude = 1.0
initial_frequency = 1.0
initial_phase = 0.0
initial_noise_mean = 0.0
initial_noise_covariance = 0.1
noise_g = None

t = np.linspace(0, 10, 1000)
sampling_frequency = 1/(t[1]-t[0])

plot = figure(title="Гармонійний сигнал зі шумом", x_axis_label="Час", y_axis_label="Амплітуда", width=800, height=400)

pure_harmonic_line = plot.line(t, harmonic(t, initial_amplitude, frequency=initial_frequency, phase=initial_phase),
                               line_width=2, color="violet", line_dash="dashed", legend_label='Чиста гармоніка')
harmonic_with_noise_line = plot.line(t, harmonic_with_noise(t, initial_amplitude, frequency=initial_frequency,
                                                            phase=initial_phase, noise_mean=initial_noise_mean,
                                                            noise_covariance=initial_noise_covariance),   
                                     line_width=2, color="red", legend_label='Гармоніка з шумом')

filtered_signal = moving_avg(harmonic_with_noise_line.data_source.data['y'], sampling_frequency)
l_filtered = plot.line(t, filtered_signal, line_width=2, color='green', legend_label='Відфільтрований сигнал')

s_amplitude = Slider(title="Амплітуда", value=initial_amplitude, start=0.1, end=10.0, step=0.1)
s_frequency = Slider(title="Частота", value=initial_frequency, start=0.1, end=10.0, step=0.1)
s_phase = Slider(title="Фаза", value=initial_phase, start=0.0, end=2 * np.pi, step=0.1)
s_noise_mean = Slider(title="Амплітуда шуму", value=initial_noise_mean, start=-1.0, end=1.0, step=0.1)
s_noise_covariance = Slider(title="Диспепрсія шуму", value=initial_noise_covariance, start=0.0, end=1.0, step=0.1)
s_cutoff_frequency = Slider(title="Розмір", value=3, start=1, end=35, step=1)

ck_show_noise = CheckboxGroup(labels=['Показати шум'], active=[0])

button_regenerate_noise = Button(label='Згенерувати новий шум')
button_random_param = Button(label='Випадкові параметри')
button_reset = Button(label='Скинути')

def regenerate_noise():
    s_noise_mean.value = np.random.uniform(-1.0, 1.0)
    s_noise_covariance.value = np.random.uniform(0.0, 1.0)
    update(None, None, None)
    show_noise = 0 in ck_show_noise.active 
    harmonic_with_noise_line.data_source.data['y'] = harmonic_with_noise(t, s_amplitude.value, s_frequency.value,
                                                                         s_phase.value, s_noise_mean.value,
                                                                         s_noise_covariance.value, show_noise, noise_g)
    filtered_signal = moving_avg(harmonic_with_noise_line.data_source.data['y'], s_cutoff_frequency.value)
    l_filtered.data_source.data['y'] = filtered_signal

def update(attrname, old, new):
    amplitude = s_amplitude.value
    frequency = s_frequency.value
    phase = s_phase.value
    noise_mean = s_noise_mean.value
    noise_covariance = s_noise_covariance.value

    global initial_noise_mean, initial_noise_covariance, noise_g

    if initial_noise_covariance != noise_covariance or initial_noise_mean != noise_mean:
        initial_noise_mean = noise_mean
        initial_noise_covariance = noise_covariance
        noise_g = create_noise(t, noise_mean, noise_covariance)

    show_noise = 0 in ck_show_noise.active  
    harmonic_with_noise_line.data_source.data['y'] = harmonic_with_noise(t, amplitude, frequency, phase, noise_mean,
                                                                         noise_covariance, show_noise, noise_g)

    filtered_signal = moving_avg(harmonic_with_noise_line.data_source.data['y'], s_cutoff_frequency.value)
    l_filtered.data_source.data['y'] = filtered_signal

    pure_harmonic_line.data_source.data['y'] = harmonic(t, amplitude, frequency, phase)

def random_params():
    s_amplitude.value = np.random.uniform(0.1, 10.0)
    s_frequency.value = np.random.uniform(0.1, 10.0)
    s_phase.value = np.random.uniform(0.0, 2 * np.pi)
    s_noise_mean.value = np.random.uniform(-1.0, 1.0)
    s_noise_covariance.value = np.random.uniform(0.0, 1.0)
    update(None, None, None)

def reset_params():
    s_amplitude.value = initial_amplitude
    s_frequency.value = initial_frequency
    s_phase.value = initial_phase
    s_noise_mean.value = initial_noise_mean
    s_noise_covariance.value = initial_noise_covariance
    update(None, None, None)

s_amplitude.on_change('value', update)
s_frequency.on_change('value', update)
s_phase.on_change('value', update)
s_noise_mean.on_change('value', update)
s_noise_covariance.on_change('value', update)
s_cutoff_frequency.on_change('value', update)
ck_show_noise.on_change('active', update)
button_regenerate_noise.on_click(regenerate_noise)
button_random_param.on_click(random_params)
button_reset.on_click(reset_params)

layout = column(
    plot,
    row(s_amplitude, s_frequency, s_phase),
    row(s_noise_mean, s_noise_covariance, s_cutoff_frequency),
    row(ck_show_noise, button_regenerate_noise, button_random_param, button_reset),
    sizing_mode='stretch_width',
    align='center'
)

curdoc().add_root(layout)
